In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
import base64
from datetime import date

# PDF बनाउन weasyprint लाइब्रेरी प्रयास गर्ने
try:
    from weasyprint import HTML as weasyHTML
    WEASYPRINT_AVAILABLE = True
except ImportError:
    WEASYPRINT_AVAILABLE = False


# --- Dynamic Widget Sections ---
# यी VBox हरूमा dynamic रूपमा थपिने विजेटहरू (जस्तै: शिक्षा, अनुभव) रहनेछन्।
education_entries_box = widgets.VBox([])
work_entries_box = widgets.VBox([])
language_entries_box = widgets.VBox([])
# यी list हरूमा विजेट object हरू राखिनेछन् ताकि पछि तिनीहरूको value पढ्न सकियोस्।
education_widgets_list = []
work_widgets_list = []
language_widgets_list = []


# --- Function to Add New Entries ---
def add_education_entry(b):
    new_entry = widgets.HBox([
        widgets.Text(placeholder='डिग्री (जस्तै: SEE, +2, Bachelor)', layout={'width': '25%'}),
        widgets.Text(placeholder='शिक्षण संस्थाको नाम', layout={'width': '40%'}),
        widgets.Text(placeholder='उत्तीर्ण भएको साल (जस्तै: 2075 B.S.)', layout={'width': '25%'})
    ])
    education_widgets_list.append(new_entry)
    education_entries_box.children = tuple(list(education_entries_box.children) + [new_entry])

def add_work_entry(b):
    new_entry = widgets.HBox([
        widgets.Text(placeholder='कम्पनीको नाम', layout={'width': '30%'}),
        widgets.Text(placeholder='पद (Role)', layout={'width': '30%'}),
        widgets.Text(placeholder='अवधि (जस्तै: 2076 - 2078)', layout={'width': '30%'})
    ])
    work_widgets_list.append(new_entry)
    work_entries_box.children = tuple(list(work_entries_box.children) + [new_entry])

def add_language_entry(b):
    new_entry = widgets.HBox([
        widgets.Text(placeholder='भाषा (जस्तै: नेपाली, अंग्रेजी)', layout={'width': '40%'}),
        widgets.Dropdown(options=['साधारण', 'राम्रो', 'उत्कृष्ट'], value='राम्रो', layout={'width': '40%'})
    ])
    language_widgets_list.append(new_entry)
    language_entries_box.children = tuple(list(language_entries_box.children) + [new_entry])

# --- CV Generator and Downloader ---
cv_output = widgets.Output()
download_output = widgets.Output()
cv_html_content = "" # ग्लोबल चर जसले CV को HTML राख्छ

# मुख्य विजेटहरू बनाउने
style = {'description_width': 'initial'}
# Personal Details
name = widgets.Text(description="पूरा नाम:", style=style)
address = widgets.Text(description="स्थायी ठेगाना:", style=style)
phone = widgets.Text(description="फोन नम्बर:", style=style)
email = widgets.Text(description="इमेल:", style=style)
dob = widgets.DatePicker(description="जन्म मिति:", value=date(2000, 1, 1), style=style)
sex = widgets.Dropdown(options=['पुरुष', 'महिला', 'अन्य'], description="लिङ्ग:", style=style)
nationality = widgets.Text(value="नेपाली", description="राष्ट्रियता:", style=style)
citizenship_no = widgets.Text(description="नागरिकता नं:", style=style)
marital_status = widgets.Dropdown(options=['विवाहित', 'अविवाहित'], description="वैवाहिक स्थिति:", style=style)
objective = widgets.Textarea(description="उद्देश्य (Objective):", placeholder="तपाईंको करियरको उद्देश्य लेख्नुहोस्...", layout={'height': '100px'})
photo_uploader = widgets.FileUpload(accept='image/*', description="फोटो अपलोड गर्नुहोस्", style=style)

# Add Buttons
add_education_button = widgets.Button(description="थप शिक्षा विवरण", icon='plus', button_style='info')
add_work_button = widgets.Button(description="थप कार्य अनुभव", icon='plus', button_style='info')
add_language_button = widgets.Button(description="थप भाषा", icon='plus', button_style='info')

# Button Actions
add_education_button.on_click(add_education_entry)
add_work_button.on_click(add_work_entry)
add_language_button.on_click(add_language_entry)

# Generate and Download Buttons
generate_button = widgets.Button(description="सिभी बनाउनुहोस् (Generate CV)", button_style='success', icon='check')
download_pdf_button = widgets.Button(description="PDF डाउनलोड गर्नुहोस्", button_style='danger', icon='download', disabled=not WEASYPRINT_AVAILABLE)

def generate_cv_html(b):
    global cv_html_content
    photo_html = ""
    if photo_uploader.value:
        # Get the first uploaded file's content from the tuple
        # ===== यहाँ त्रुटि सच्याइएको छ =====
        photo_data = photo_uploader.value[0]['content'] 
        b64_photo = base64.b64encode(photo_data).decode('utf-8')
        photo_html = f'<img src="data:image/png;base64,{b64_photo}" alt="Photo" class="cv-photo">'

    # Dynamic sections
    edu_html = "".join([f"""
        <div class="item">
            <p><b>- {entry.children[0].value}</b></p>
            <p><i>&nbsp;&nbsp; शिक्षण संस्था:</i> {entry.children[1].value}</p>
            <p><i>&nbsp;&nbsp; साल:</i> {entry.children[2].value}</p>
        </div>
    """ for entry in education_widgets_list if entry.children[0].value])

    work_html = "".join([f"""
        <div class="item">
            <p><b>- पद: {entry.children[1].value}</b></p>
            <p><i>&nbsp;&nbsp; कम्पनी:</i> {entry.children[0].value}</p>
            <p><i>&nbsp;&nbsp; अवधि:</i> {entry.children[2].value}</p>
        </div>
    """ for entry in work_widgets_list if entry.children[0].value])
    
    lang_html = "".join([f"""
        <p>- {entry.children[0].value} <i>({entry.children[1].value})</i></p>
    """ for entry in language_widgets_list if entry.children[0].value])

    cv_html_content = f"""
    <html>
    <head>
        <meta charset="UTF-8">
        <style>
            body {{ font-family: 'DejaVu Sans', sans-serif; line-height: 1.6; color: #333; }}
            .cv-container {{ border: 1px solid #ccc; padding: 30px; margin: 20px; background: #fff; }}
            .cv-header {{ text-align: center; border-bottom: 2px solid #333; padding-bottom: 15px; position: relative; }}
            .cv-photo {{ width: 120px; height: 120px; border-radius: 50%; border: 3px solid #ddd; position: absolute; top: 10px; right: 10px; object-fit: cover; }}
            .cv-header h1 {{ margin: 0; }}
            .cv-section {{ margin-top: 20px; }}
            .cv-section h3 {{ border-bottom: 1px solid #ddd; padding-bottom: 5px; color: #0056b3; }}
            .item {{ margin-bottom: 10px; }}
            p {{ margin: 5px 0; }}
        </style>
    </head>
    <body>
        <div class="cv-container">
            <div class="cv-header">
                {photo_html}
                <h1>{name.value}</h1>
                <p>{address.value} | फोन: {phone.value} | इमेल: {email.value}</p>
            </div>

            <div class="cv-section">
                <h3>उद्देश्य (Objective)</h3>
                <p>{objective.value}</p>
            </div>

            <div class="cv-section">
                <h3>व्यक्तिगत विवरण (Personal Details)</h3>
                <p><b>जन्म मिति:</b> {dob.value}</p>
                <p><b>लिङ्ग:</b> {sex.value}</p>
                <p><b>राष्ट्रियता:</b> {nationality.value}</p>
                <p><b>नागरिकता नम्बर:</b> {citizenship_no.value}</p>
                <p><b>वैवाहिक स्थिति:</b> {marital_status.value}</p>
            </div>

            <div class="cv-section">
                <h3>शैक्षिक योग्यता (Academic Qualification)</h3>
                {edu_html if edu_html else "<p>विवरण उपलब्ध छैन।</p>"}
            </div>
            
            <div class="cv-section">
                <h3>कार्य अनुभव (Work Experience)</h3>
                {work_html if work_html else "<p>विवरण उपलब्ध छैन।</p>"}
            </div>

            <div class="cv-section">
                <h3>भाषा दक्षता (Language Proficiency)</h3>
                {lang_html if lang_html else "<p>विवरण उपलब्ध छैन।</p>"}
            </div>
        </div>
    </body>
    </html>
    """
    with cv_output:
        cv_output.clear_output(wait=True)
        display(HTML(cv_html_content))
    with download_output:
        download_output.clear_output()

def download_pdf(b):
    global cv_html_content
    if not cv_html_content:
        with download_output:
            download_output.clear_output(wait=True)
            print("कृपया पहिले 'सिभी बनाउनुहोस्' बटनमा क्लिक गर्नुहोस्।")
        return
    
    pdf_bytes = weasyHTML(string=cv_html_content).write_pdf()
    b64_pdf = base64.b64encode(pdf_bytes).decode('utf-8')
    
    file_name = f"CV_{name.value.replace(' ', '_')}.pdf"
    
    download_link_html = f'<a download="{file_name}" href="data:application/pdf;base64,{b64_pdf}" target="_blank">यहाँ क्लिक गरेर PDF डाउनलोड गर्नुहोस्</a>'
    
    with download_output:
        download_output.clear_output(wait=True)
        display(HTML(download_link_html))

generate_button.on_click(generate_cv_html)
download_pdf_button.on_click(download_pdf)

# --- Layout ---
# सबै विजेटहरूलाई Accordion मा राख्ने ताकि UI सफा देखियोस्
personal_details = widgets.VBox([name, address, phone, email, dob, sex, nationality, citizenship_no, marital_status])
objective_box = widgets.VBox([objective])
photo_box = widgets.VBox([photo_uploader])
education_box = widgets.VBox([education_entries_box, add_education_button])
work_box = widgets.VBox([work_entries_box, add_work_button])
language_box = widgets.VBox([language_entries_box, add_language_button])

accordion = widgets.Accordion(children=[
    personal_details, 
    objective_box, 
    photo_box,
    education_box, 
    work_box, 
    language_box
])

accordion.set_title(0, '१. व्यक्तिगत विवरण')
accordion.set_title(1, '२. उद्देश्य (Objective)')
accordion.set_title(2, '३. फोटो')
accordion.set_title(3, '४. शैक्षिक योग्यता')
accordion.set_title(4, '५. कार्य अनुभव')
accordion.set_title(5, '६. भाषा')

# Final Display
print("============== सिभी बनाउने फारम ==============")
if not WEASYPRINT_AVAILABLE:
    print("\nचेतावनी: 'weasyprint' लाइब्रेरी भेटिएन। PDF डाउनलोड सुविधा उपलब्ध छैन।")
    print("स्थापना गर्न: pip install weasyprint")

# सुरुमा केही खाली entry हरू थप्ने
add_education_entry(None)
# add_work_entry(None) # Initially not adding work and language for a cleaner look
# add_language_entry(None)

display(accordion, widgets.HBox([generate_button, download_pdf_button]), download_output, cv_output)

============== सिभी बनाउने फारम ==============

चेतावनी: 'weasyprint' लाइब्रेरी भेटिएन। PDF डाउनलोड सुविधा उपलब्ध छैन।
स्थापना गर्न: pip install weasyprint


Accordion(children=(VBox(children=(Text(value='', description='पूरा नाम:', style=TextStyle(description_width='…

Output()

Output()